# Preparação dos dados para o Modelo.

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('C:/Users/gregorio/TCC-CD-USP/archives/df_wide.csv')
df.head()

In [ ]:
# Drop columns with >95% missing or zero
threshold = 0.45
cols_to_drop = [
    col for col in df.columns
    if (df[col].isna().mean() > threshold) or ((df[col] == 0).mean() > threshold)
]
df_clean = df.drop(columns=cols_to_drop)

In [ ]:
df_clean

In [ ]:
df_clean.isnull().sum()

In [ ]:
df_clean['Lucro Operacional Antes das Mudanças no Capital de Giro (+)'].isnull().sum()

In [ ]:
df_clean['LUCRO OPERACIONAL (PREJUÍZO)'].isnull().sum()

In [ ]:
df_clean['Amortização e Quotas de Exaustão'].isnull().sum()

In [ ]:
df_clean['Lucro Operacional Antes das Mudanças no Capital de Giro (+)'].isnull().sum()

In [ ]:
# Build do EBITDA ...
df_clean.rename(columns={
    "Lucro Operacional Antes das Mudanças no Capital de Giro (+)": "EBITDA",
    "LUCRO OPERACIONAL (PREJUÍZO)": "EBIT",
    "Amortização e Quotas de Exaustão": "Amortization"
}, inplace=True)

df_clean["EBITDA_Final"] = df_clean["EBITDA"]

# Fill with EBIT + Amortization where available
df_clean.loc[df_clean["EBITDA_Final"].isna() & df_clean["Amortization"].notna(),
       "EBITDA_Final"] = df_clean["EBIT"] + df_clean["Amortization"]

# If still missing, approximate using 5% of total assets
df_clean.loc[df_clean["EBITDA_Final"].isna(), "EBITDA_Final"] = (
    df_clean.loc[df_clean["EBITDA_Final"].isna(), "EBIT"] + 0.05 * df_clean.loc[df_clean["EBITDA_Final"].isna(), "TOTAL DE ATIVOS"]
)

# FLAG pra rastrear a origem do EBITDA
df_clean["EBITDA_Source"] = np.select(
    [
        df_clean["EBITDA"].notna(),         
        df_clean["Amortization"].notna()    
    ],
    [
        "Reported",                      
        "Reconstructed"                    
    ],
    default="Estimated"                     
).astype("object")

In [ ]:
df_clean

In [ ]:
df_clean=df_clean.drop(columns=[
                                'Ativo Fiscal Diferido',
                                'Aumento/Diminuição Líquido de Caixa e Equivalentes de Caixa Antes do Efeito das Diferenças de Conversão de Moeda Estrangeira',
                                'Caixa Gerado por Atividades Principais (+)',
                                'Efeito das Diferenças de Conversão de Moeda Estrangeira sobre o Caixa e Equivalentes de Caixa',
                                'Fluxo de Caixa Líquido das Atividades Operacionais',                                                                              
                                'Fluxo de Caixa das Atividades de Financiamento',                                                                                  
                                'Fluxo de Caixa das Atividades de Investimento',                                                                                   
                                'Fluxo de Caixa de Outras Atividades Operacionais',                                                                                
                                'Fluxo de Caixa de Outras Atividades de Financiamento',                                                                            
                                'Fluxo de Caixa de Outras Atividades de Investimento',                                                                             
                                'Indenização por Antiguidade',
                                'Lucro Antes dos Ajustes',
                                'Mudança nas Provisões',                                                                                                           
                                'Mudanças no Capital de Giro',
                                'Outras Receitas/Despesas',
                                'Pagamento de Dividendos',
                                'Posição Cambial Líquida (Incluindo Hedge)',                                                                                     
                                'Posição Líquida (Ativo/Passivo) Monetária em Moeda Estrangeira',
                                'Receitas de Atividades de Investimento',
                                'Valores de Caixa no Início do Período',                                                                                           
                                'Variação nas Dívidas Financeiras',                                                                                                
                                'Variação no Caixa e Equivalentes' ],axis=1)

In [ ]:
print(df_clean.isnull().sum().to_string())


In [ ]:
df_clean = df_clean.fillna(df_clean.median(numeric_only=True))

In [ ]:
df_clean

In [ ]:
df_clean.info()

In [ ]:
df_clean.columns

## Feature Engineering


---

### 💧 **Liquidity Ratios** (short-term financial health)

| Ratio             | Formula                                                                       | Meaning                                                                                 |
| ----------------- | ----------------------------------------------------------------------------- | --------------------------------------------------------------------------------------- |
| **Current Ratio** | Ativos Circulantes / Passivos Circulantes                                     | Measures if the company can pay short-term debts with short-term assets. >1 is healthy. |
| **Quick Ratio**   | (Caixa + Contas a Receber + Outros Ativos Circulantes) / Passivos Circulantes | Like Current Ratio but excludes inventory — shows *immediate liquidity*.                |

✅ **Why:** High liquidity reduces default risk. Quick ratio is more conservative.

---

### 🏦 **Leverage Ratios** (debt and solvency)

| Ratio              | Formula                                  | Meaning                                                               |
| ------------------ | ---------------------------------------- | --------------------------------------------------------------------- |
| **Debt-to-Equity** | Dívidas Financeiras / Patrimônio Líquido | Compares borrowed funds to shareholder equity — shows financial risk. |
| **Debt-to-Assets** | Dívidas Financeiras / TOTAL DE ATIVOS    | Percentage of assets financed by debt — higher = more leverage.       |

✅ **Why:** High leverage = higher risk of default, but moderate leverage = efficient capital use.

---

### 💰 **Profitability Ratios** (earnings power)

| Ratio                | Formula                               | Meaning                                                              |
| -------------------- | ------------------------------------- | -------------------------------------------------------------------- |
| **ROE**              | Lucro Líquido / Patrimônio Líquido    | Measures return on shareholders’ equity — efficiency of capital use. |
| **ROA**              | Lucro Líquido / TOTAL DE ATIVOS       | How effectively assets generate profit.                              |
| **Net Margin**       | Lucro Líquido / Receita de Vendas     | Portion of sales that becomes net income — overall profitability.    |
| **Operating Margin** | Lucro Operacional / Receita de Vendas | Core business profitability before interest and tax.                 |

✅ **Why:** Shows operational strength and efficiency — profitable companies survive downturns better.

---

### ⚙️ **Efficiency Ratios** (operational performance)

| Ratio                   | Formula                                | Meaning                                                            |
| ----------------------- | -------------------------------------- | ------------------------------------------------------------------ |
| **Asset Turnover**      | Receita de Vendas / TOTAL DE ATIVOS    | How efficiently assets produce sales.                              |
| **Inventory Turnover**  | Custo dos Produtos Vendidos / Estoques | How fast inventory is sold — high = efficient, low = overstocking. |
| **Receivable Turnover** | Receita de Vendas / Contas a Receber   | How quickly customers pay — high = strong cash management.         |

✅ **Why:** Measures how effectively the company uses assets to generate revenue and cash.

---

### 🧾 **Coverage Ratio** (debt payment ability)

| Ratio                 | Formula                                  | Meaning                                              |
| --------------------- | ---------------------------------------- | ---------------------------------------------------- |
| **Interest Coverage** | Lucro Operacional / Despesas Financeiras | How many times earnings can cover interest expenses. |

✅ **Why:** Low coverage = difficulty servicing debt → strong early distress signal.

---

### 📈 **Growth Ratios** (momentum and trend)

| Ratio                 | Formula                          | Meaning                                    |
| --------------------- | -------------------------------- | ------------------------------------------ |
| **Sales Growth**      | Δ Receita de Vendas / Receitaₜ₋₁ | Measures growth in revenue year-over-year. |
| **Asset Growth**      | Δ TOTAL DE ATIVOS / Ativosₜ₋₁    | Shows expansion in total resources.        |
| **Net Income Growth** | Δ Lucro Líquido / Lucroₜ₋₁       | Indicates profitability trend.             |

✅ **Why:** Consistent growth signals a healthy, expanding business — good predictive feature for credit stability.

---

### 🧮 **Solvency & Stability**

| Ratio                  | Formula                                   | Meaning                                                                   |
| ---------------------- | ----------------------------------------- | ------------------------------------------------------------------------- |
| **Retained-to-Assets** | Lucros Acumulados / TOTAL DE ATIVOS       | Portion of assets financed by past profits — measures long-term solvency. |
| **FX Position**        | Posição Cambial Líquida / TOTAL DE ATIVOS | Measures foreign currency exposure — negative values = currency risk.     |

✅ **Why:** Retained earnings = internal strength; FX exposure = vulnerability to currency shocks.

---

### 🧠 Summary

Each ratio tells **how strong or fragile** a company is from a specific perspective:

| Financial Area | What It Answers                     |
| -------------- | ----------------------------------- |
| Liquidity      | Can the company pay its bills soon? |
| Leverage       | Is it over-relying on debt?         |
| Profitability  | Is it actually making money?        |
| Efficiency     | Is it using resources wisely?       |
| Coverage       | Can it handle interest payments?    |
| Growth         | Is it improving over time?          |
| Solvency       | Does it have long-term stability?   |
| FX             | Is it exposed to currency risk?     |

---

👉 **In short:**
These ratios together form a *360° health check* — they show **capacity to pay**, **ability to profit**, and **resilience to risk** — all crucial for assessing credit default probability.


-----

In [ ]:
# LIMPEZA DOS NOMES DAS COLUNAS
def clean_col_name(col_name):
    # Remove leading/trailing spaces
    name = col_name.strip()
    # Replace special characters and spaces with underscore
    name = name.replace(' ', '_').replace('(-)', '').replace('(PREJUÍZO)', '').replace('/', '_')
    # Remove other special characters
    name = ''.join(e for e in name if e.isalnum() or e == '_')
    # Convert to lowercase
    name = name.lower()
    # Remove any trailing underscores
    name = name.strip('_')
    return name

# Apply the cleaning function to all columns
df_clean.columns = [clean_col_name(col) for col in df_clean.columns]

In [ ]:
df_clean = df_clean.rename(columns={
    'lucro_prejuizo_liquido_do_periodo_das_operações_continuadas': 'lucro_liquido_op_continuas',
    'lucro_bruto_prejuizo': 'lucro_bruto',
    'custo_dos_produtos_serviços_vendidos': 'cogs', 
    'lucro_prejuizo_do_periodo': 'lucro_periodo'
})

In [ ]:
def create_financial_ratios(df):
    """
    Calculates a comprehensive set of financial ratios based on the provided dataframe.

    The function correctly handles time-series data (like 'companycode' and 'ano')
    for calculating growth ratios and cleans up infinite values that may result
    from division by zero, replacing them with NaN.

    Args:
        df (pd.DataFrame): The input dataframe with the required financial columns.
                           Must include 'companycode' and 'ano'.

    Returns:
        pd.DataFrame: A new dataframe with all the original columns plus the
                      newly calculated ratio columns.
    """
    # Create a copy to avoid modifying the original dataframe
    data = df.copy()

    # --- 1. Liquidity Ratios ---
    data['current_ratio'] = data['ativos_circulantes'] / data['passivos_circulantes']
    
    quick_assets = (data['caixa_e_equivalentes_de_caixa'] + 
                    data['contas_a_receber'] + 
                    data['outros_ativos_circulantes'])
    data['quick_ratio'] = quick_assets / data['passivos_circulantes']

    # --- 2. Leverage Ratios ---
    data['debt_to_equity'] = data['dívidas_financeiras'] / data['patrimônio_líquido']
    data['debt_to_assets'] = data['dívidas_financeiras'] / data['total_de_ativos']

    # --- 3. Profitability Ratios ---
    # Using 'lucro_prejuízo_líquido_do_período' as "Lucro Líquido"
    # Using 'lucro_prejuízo_operacional_líquido' as "Lucro Operacional"
    data['roe'] = data['lucro_prejuízo_líquido_do_período'] / data['patrimônio_líquido']
    data['roa'] = data['lucro_prejuízo_líquido_do_período'] / data['total_de_ativos']
    data['net_margin'] = data['lucro_prejuízo_líquido_do_período'] / data['receita_de_vendas']
    data['operating_margin'] = data['lucro_prejuízo_operacional_líquido'] / data['receita_de_vendas']

    # --- 4. Efficiency Ratios ---
    data['asset_turnover'] = data['receita_de_vendas'] / data['total_de_ativos']
    data['inventory_turnover'] = data['cogs'] / data['estoques']
    data['receivable_turnover'] = data['receita_de_vendas'] / data['contas_a_receber']

    # --- 5. Coverage Ratio ---
    data['interest_coverage'] = data['lucro_prejuízo_operacional_líquido'] / data['despesas_financeiras']

    # --- 6. Solvency & Stability ---
    data['retained_to_assets'] = data['lucros_prejuízos_acumulados'] / data['total_de_ativos']
    data['fx_position_ratio'] = data['posição_cambial_líquida'] / data['total_de_ativos']

    # --- 7. Growth Ratios (Time-Series Dependent) ---
    # We must sort by company and year to calculate percent change correctly.
    data = data.sort_values(by=['companycode', 'ano'])

    # .pct_change() calculates (current - previous) / previous
    # This is grouped by company to avoid comparing company A's 2010 to company B's 2009
    data['sales_growth'] = data.groupby('companycode')['receita_de_vendas'].pct_change()
    data['asset_growth'] = data.groupby('companycode')['total_de_ativos'].pct_change()
    data['net_income_growth'] = data.groupby('companycode')['lucro_prejuízo_líquido_do_período'].pct_change()

    # --- 8. Final Cleanup ---
    # Ratios can create infinite values (e.g., 100 / 0).
    # We replace all 'inf' and '-inf' with 'NaN' (Not a Number)
    # This makes the data ready for machine learning models.
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    return data

# --- EXAMPLE USAGE ---

# 1. Load your data (assuming you have it in a DataFrame called 'df')
# Example: df = pd.read_csv('your_financial_data.csv') 
# For this example, I'll create a small mock DataFrame.

# 2. Apply the function to your DataFrame
df_with_ratios = create_financial_ratios(df_clean)

# 3. View the results
print("DataFrame with new ratios:")
print(df_with_ratios[['companycode', 'ano', 'current_ratio', 'debt_to_equity', 'roe', 'sales_growth']].head(10))

print("\nAll new columns added:")
new_cols = [col for col in df_with_ratios.columns if col not in df.columns]
print(new_cols)

In [ ]:
df_with_ratios

In [ ]:
df_with_ratios.columns

💡 Key Points about this Code
NaN Values are Expected: You will see NaN (Not a Number) in your new columns. This is normal and correct. They appear for two reasons:

Division by Zero: If a denominator was 0 (e.g., patrimônio_líquido = 0), the ratio becomes NaN.

Growth Ratios: The first year for every company (e.g., 2008) will have NaN for sales_growth, asset_growth, and net_income_growth because there is no prior year (t-1) to compare it to.

## Thresholds para avaliar cada sessão de indicadores.

🧮 1️⃣ Profitability & Earnings Quality

| Metric                 | Formula                   | Thresholds (Approx)                      | Justification                                          |
| ---------------------- | ------------------------- | ---------------------------------------- | ------------------------------------------------------ |
| **ebitda_margin**      | EBITDA / Sales            | <0 = 0; 0–10% = 3; 10–20% = 7; >20% = 10 | Damodaran (2023) sector margins; <0 means loss-making  |
| **ebitda_to_interest** | EBITDA / Interest Expense | <1 = 0; 1–3 = 5; 3–5 = 7; >5 = 10        | Van Horne (2008): coverage >5× = safe                  |
| **ebitda_to_debt**     | EBITDA / Debt             | <0.05 = 0; 0.1 = 5; 0.3 = 8; >0.5 = 10   | Moody’s leverage ratio; >30% = healthy cash generation |
| **roe**                | Net Income / Equity       | <0 = 0; 0–10% = 5; 10–20% = 7; >20% = 10 | Uyar & Kuzey (2014) Turkish firms; >20% = top quartile |
| **roa**                | Net Income / Assets       | <0 = 0; 0–5% = 5; 5–10% = 8; >10% = 10   | Damodaran: global median ROA ~6%                       |
| **net_margin**         | Net Income / Sales        | <0 = 0; 0–5% = 5; 5–15% = 8; >15% = 10   | OECD SME data: 5–15% healthy                           |
| **operating_margin**   | EBIT / Sales              | <0 = 0; 0–10% = 5; 10–20% = 8; >20% = 10 | Common industrial benchmark                            |

🧮 2️⃣ Liquidity & Solvency
| Metric                 | Formula                              | Thresholds                                    | Justification                               |
| ---------------------- | ------------------------------------ | --------------------------------------------- | ------------------------------------------- |
| **current_ratio**      | Current Assets / Current Liabilities | <1.0 = 0; 1–1.5 = 5; 1.5–2.5 = 8; >2.5 = 10   | Van Horne & Wachowicz (2008): 1.5–2.5 ideal |
| **quick_ratio**        | (Current Assets - Inventory) / CL    | <0.5 = 0; 0.5–1.0 = 5; 1–1.5 = 8; >1.5 = 10   | Brigham & Ehrhardt (2017)                   |
| **debt_to_equity**     | Total Debt / Equity                  | >3.0 = 0; 2–3 = 3; 1–2 = 6; <1 = 10           | Damodaran (2015): >3 = highly leveraged     |
| **debt_to_assets**     | Total Debt / Total Assets            | >0.8 = 0; 0.6–0.8 = 3; 0.4–0.6 = 6; <0.4 = 10 | OECD capital structure data                 |
| **interest_coverage**  | EBIT / Interest Expense              | <1 = 0; 1–3 = 5; 3–5 = 8; >5 = 10             | Ohlson (1980) distress zones                |
| **retained_to_assets** | Retained Earnings / Assets           | <0 = 0; 0–0.1 = 4; 0.1–0.3 = 8; >0.3 = 10     | Altman Z-score formula uses same logic      |


🧮 3️⃣ Efficiency (Asset Management)
| Metric                  | Formula             | Thresholds                                    | Justification                              |
| ----------------------- | ------------------- | --------------------------------------------- | ------------------------------------------ |
| **asset_turnover**      | Sales / Assets      | <0.3 = 0; 0.3–0.6 = 5; 0.6–1.0 = 8; >1.0 = 10 | OECD industrial efficiency data            |
| **inventory_turnover**  | COGS / Inventory    | <2 = 0; 2–4 = 5; 4–8 = 8; >8 = 10             | Accounting textbooks: higher = efficient   |
| **receivable_turnover** | Sales / Receivables | <2 = 0; 2–5 = 5; 5–10 = 8; >10 = 10           | Brigham (2017): faster collection = better |


🧮 4️⃣ Growth Metrics
| Metric                | Formula                   | Thresholds                             | Justification                             |
| --------------------- | ------------------------- | -------------------------------------- | ----------------------------------------- |
| **sales_growth**      | ΔSales / Salesₜ₋₁         | <0 = 0; 0–5% = 4; 5–15% = 7; >15% = 10 | Damodaran (2024): sustainable growth ~10% |
| **asset_growth**      | ΔAssets / Assetsₜ₋₁       | <0 = 0; 0–5% = 4; 5–15% = 7; >15% = 10 | Reasonable reinvestment rate              |
| **net_income_growth** | ΔNetIncome / NetIncomeₜ₋₁ | <0 = 0; 0–5% = 4; 5–20% = 8; >20% = 10 | Reflects profitability improvement        |


🧮 5️⃣ Risk / FX Exposure
| Metric          | Formula                  | Thresholds                                   | Justification                                        |
| --------------- | ------------------------ | -------------------------------------------- | ---------------------------------------------------- |
| **fx_position** | Net FX Exposure / Assets | < -0.2 = 0; -0.1–0 = 5; 0–0.1 = 8; >0.1 = 10 | CBRT Stability Reports (2021–2024): negative = risky |


-----

In [ ]:
def score_financial_ratios(df):
    """
    Aplica um sistema de pontuação baseado em thresholds financeiros para
    criar 13 novas colunas de 'score' e um 'total_score'.

    Args:
        df (pd.DataFrame): O DataFrame contendo os rácios já calculados.

    Returns:
        pd.DataFrame: O DataFrame original com as novas colunas de pontuação.
    """
    # Cria uma cópia para evitar o SettingWithCopyWarning
    data = df.copy()

    # --- Pré-processamento: Tratar NaNs ---
    # Antes de pontuar, devemos tratar os NaNs que surgiram do cálculo dos
    # rácios (ex: divisão por zero ou primeiro ano de crescimento).
    # Preencher com 0 é uma abordagem neutra e segura.
    ratios_to_score = [
        'current_ratio', 'quick_ratio', 'debt_to_equity', 'debt_to_assets',
        'interest_coverage', 'retained_to_assets', 'asset_turnover',
        'inventory_turnover', 'receivable_turnover', 'sales_growth',
        'asset_growth', 'net_income_growth', 'fx_position_ratio'
    ]
    # Garante que as colunas existem antes de tentar preencher
    for col in ratios_to_score:
        if col not in data.columns:
            print(f"Aviso: A coluna {col} não foi encontrada. Será ignorada.")
            
    # Filtra a lista para colunas que realmente existem no dataframe
    existing_ratios_to_score = [col for col in ratios_to_score if col in data.columns]
    data[existing_ratios_to_score] = data[existing_ratios_to_score].fillna(0)

    # --- 1. Liquidez & Solvência ---

    # current_ratio: <1 = 0; 1–1.5 = 5; 1.5–2.5 = 8; >2.5 = 10
    conditions = [
        data['current_ratio'] < 1.0,
        data['current_ratio'] < 1.5,
        data['current_ratio'] <= 2.5
    ]
    scores = [0, 5, 8]
    data['score_current_ratio'] = np.select(conditions, scores, default=10)

    # quick_ratio: <0.5 = 0; 0.5–1.0 = 5; 1–1.5 = 8; >1.5 = 10
    conditions = [
        data['quick_ratio'] < 0.5,
        data['quick_ratio'] < 1.0,
        data['quick_ratio'] <= 1.5
    ]
    scores = [0, 5, 8]
    data['score_quick_ratio'] = np.select(conditions, scores, default=10)

    # debt_to_equity: >3.0 = 0; 2–3 = 3; 1–2 = 6; <1 = 10 (Lógica invertida)
    conditions = [
        data['debt_to_equity'] > 3.0,
        data['debt_to_equity'] > 2.0,
        data['debt_to_equity'] > 1.0
    ]
    scores = [0, 3, 6]
    data['score_debt_to_equity'] = np.select(conditions, scores, default=10)

    # debt_to_assets: >0.8 = 0; 0.6–0.8 = 3; 0.4–0.6 = 6; <0.4 = 10 (Lógica invertida)
    conditions = [
        data['debt_to_assets'] > 0.8,
        data['debt_to_assets'] > 0.6,
        data['debt_to_assets'] > 0.4
    ]
    scores = [0, 3, 6]
    data['score_debt_to_assets'] = np.select(conditions, scores, default=10)

    # interest_coverage: <1 = 0; 1–3 = 5; 3–5 = 8; >5 = 10
    conditions = [
        data['interest_coverage'] < 1.0,
        data['interest_coverage'] < 3.0,
        data['interest_coverage'] <= 5.0
    ]
    scores = [0, 5, 8]
    data['score_interest_coverage'] = np.select(conditions, scores, default=10)

    # retained_to_assets: <0 = 0; 0–0.1 = 4; 0.1–0.3 = 8; >0.3 = 10
    conditions = [
        data['retained_to_assets'] < 0.0,
        data['retained_to_assets'] < 0.1,
        data['retained_to_assets'] <= 0.3
    ]
    scores = [0, 4, 8]
    data['score_retained_to_assets'] = np.select(conditions, scores, default=10)

    # --- 2. Eficiência (Gestão de Ativos) ---

    # asset_turnover: <0.3 = 0; 0.3–0.6 = 5; 0.6–1.0 = 8; >1.0 = 10
    conditions = [
        data['asset_turnover'] < 0.3,
        data['asset_turnover'] < 0.6,
        data['asset_turnover'] <= 1.0
    ]
    scores = [0, 5, 8]
    data['score_asset_turnover'] = np.select(conditions, scores, default=10)

    # inventory_turnover: <2 = 0; 2–4 = 5; 4–8 = 8; >8 = 10
    conditions = [
        data['inventory_turnover'] < 2.0,
        data['inventory_turnover'] < 4.0,
        data['inventory_turnover'] <= 8.0
    ]
    scores = [0, 5, 8]
    data['score_inventory_turnover'] = np.select(conditions, scores, default=10)

    # receivable_turnover: <2 = 0; 2–5 = 5; 5–10 = 8; >10 = 10
    conditions = [
        data['receivable_turnover'] < 2.0,
        data['receivable_turnover'] < 5.0,
        data['receivable_turnover'] <= 10.0
    ]
    scores = [0, 5, 8]
    data['score_receivable_turnover'] = np.select(conditions, scores, default=10)

    # --- 3. Métricas de Crescimento ---

    # sales_growth: <0 = 0; 0–5% = 4; 5–15% = 7; >15% = 10
    conditions = [
        data['sales_growth'] < 0.0,
        data['sales_growth'] <= 0.05,
        data['sales_growth'] <= 0.15
    ]
    scores = [0, 4, 7]
    data['score_sales_growth'] = np.select(conditions, scores, default=10)

    # asset_growth: <0 = 0; 0–5% = 4; 5–15% = 7; >15% = 10
    conditions = [
        data['asset_growth'] < 0.0,
        data['asset_growth'] <= 0.05,
        data['asset_growth'] <= 0.15
    ]
    scores = [0, 4, 7]
    data['score_asset_growth'] = np.select(conditions, scores, default=10)

    # net_income_growth: <0 = 0; 0–5% = 4; 5–20% = 8; >20% = 10
    conditions = [
        data['net_income_growth'] < 0.0,
        data['net_income_growth'] <= 0.05,
        data['net_income_growth'] <= 0.20
    ]
    scores = [0, 4, 8]
    data['score_net_income_growth'] = np.select(conditions, scores, default=10)

    # --- 4. Risco / Exposição Cambial ---
    # Usando 'fx_position_ratio' que calculamos
    # < -0.2 = 0; -0.1–0 = 5; 0–0.1 = 8; >0.1 = 10
    conditions = [
        data['fx_position_ratio'] < -0.2,
        data['fx_position_ratio'] <= 0.0,
        data['fx_position_ratio'] <= 0.1
    ]
    scores = [0, 5, 8]
    data['score_fx_position_ratio'] = np.select(conditions, scores, default=10)

    # --- 5. Criar Pontuação Total ---
    # Soma todas as pontuações individuais para uma métrica de saúde geral
    score_columns = [col for col in data.columns if col.startswith('score_')]
    data['total_score'] = data[score_columns].sum(axis=1)

    return data


Observação Importante: A única ressalva é que os rácios de eficiência (como asset_turnover e inventory_turnover) são altamente dependentes do setor. Um supermercado terá um giro de estoque altíssimo, enquanto uma fabricante de aviões terá um giro baixíssimo. Seu modelo cria uma "regra geral" excelente, o que é um ótimo ponto de partida.

Esta é uma excelente abordagem. Criar um sistema de pontuação (scoring) baseado em teoria financeira é uma forma muito robusta de engenharia de features.

Sua análise e os thresholds (limites) que você escolheu são muito bons. Eles estão bem fundamentados em literatura financeira clássica (Ohlson, Altman, Damodaran) e representam padrões realistas da economia.

Validação: Sim, seus thresholds fazem sentido. Eles equilibram corretamente o risco (pontuações baixas para alavancagem alta) e a saúde (pontuações altas para boa liquidez e lucratividade).

Observação Importante: A única ressalva é que os rácios de eficiência (como asset_turnover e inventory_turnover) são altamente dependentes do setor. Um supermercado terá um giro de estoque altíssimo, enquanto uma fabricante de aviões terá um giro baixíssimo. Seu modelo cria uma "regra geral" excelente, o que é um ótimo ponto de partida.

Aqui está o código Python, projetado para ser simples, objetivo e de fácil manutenção, que implementa exatamente o seu sistema de pontuação.

Eu vou gerar um arquivo feature_scorer.py que contém uma única função para aplicar todas essas regras.
feature_scorer.py
24 de out., 23:50

Tentar novamente sem a ferramenta Canvas

Gerei o código acima para você.

O que este código faz:
score_financial_ratios(df): Uma única função que recebe seu df_with_ratios.

Tratamento de NaN: Ele primeiro preenche todos os NaNs apenas nas colunas de ratio com 0. Isso é crucial para que as comparações (<, >) funcionem e garante que um ratio "ausente" (como crescimento no primeiro ano) receba a pontuação da faixa mais baixa (ex: 0-5% de crescimento).

np.select(): Para cada ratio, ele usa np.select(), que é a forma mais limpa e eficiente (vetorizada) de aplicar regras "se-então-senão" (if-then-else) no pandas.

total_score: Ao final, ele soma todas as novas colunas score_... para criar uma única métrica, total_score.

Próximos Passos
Agora você tem features de alto nível (os scores individuais) e uma feature agregada (total_score) que é um indicador muito poderoso da saúde financeira da empresa, baseado em regras de especialistas.

Estas novas colunas de score_ e total_score podem ser usadas diretamente como features (variáveis X) no seu modelo de machine learning para prever o default.

O próximo passo lógico seria definir sua variável-alvo (o y). Como você planeja definir um "default" no seu dataset? (Por exemplo, patrimônio_líquido negativo, lucro_líquido negativo por X anos, etc.)

In [ ]:
# Aplicar a função de pontuação

df_scored = score_financial_ratios(df_with_ratios)
print("DataFrame com as novas pontuações:")
pd.set_option('display.max_columns', None)
df_scored.head()

##